<a href="https://colab.research.google.com/github/goguaD/finalProjectML/blob/main/Copy_of_Model_Experiment_XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

walmart_recruiting_store_sales_forecasting_path = kagglehub.competition_download('walmart-recruiting-store-sales-forecasting')

print('Data source import complete.')


Data source import complete.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"

# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

import xgboost as xgb
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error



import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install mlflow



In [ ]:
import os
os.environ['MLFLOW_TRACKING_URI'] = 'file:/content/mlruns'


In [ ]:
import mlflow
import mlflow.sklearn

In [ ]:
!pip install -q kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (2).json


{'kaggle (2).json': b'{"username":"lizikutateladze","key":"ec5a1d6b689a28840f0dfbd283842d74"}'}

In [ ]:

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c walmart-recruiting-store-sales-forecasting


walmart-recruiting-store-sales-forecasting.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip walmart-recruiting-store-sales-forecasting.zip


Archive:  walmart-recruiting-store-sales-forecasting.zip
replace features.csv.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: features.csv.zip        
replace sampleSubmission.csv.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: sampleSubmission.csv.zip  
replace stores.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: stores.csv              
replace test.csv.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test.csv.zip            
replace train.csv.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv.zip           


In [ ]:
!unzip -o train.csv.zip
!unzip -o features.csv.zip
!unzip -o test.csv.zip
!unzip -o sampleSubmission.csv.zip


Archive:  train.csv.zip
  inflating: train.csv               
Archive:  features.csv.zip
  inflating: features.csv            
Archive:  test.csv.zip
  inflating: test.csv                
Archive:  sampleSubmission.csv.zip
  inflating: sampleSubmission.csv    


In [ ]:
train    = pd.read_csv("train.csv")
features = pd.read_csv("features.csv")
stores   = pd.read_csv("stores.csv")
test     = pd.read_csv("test.csv")
train.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False


In [ ]:
train['Date']    = pd.to_datetime(train['Date'])
features['Date'] = pd.to_datetime(features['Date'])

df = (train
      .merge(features, on=['Store','Date','IsHoliday'], how='left')
      .merge(stores,   on='Store',                       how='left')
      .sort_values(['Store','Dept','Date'])
      .reset_index(drop=True))

print(df.shape)
df.head()


(421570, 16)


,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size
0,1,1,2010-02-05,24924.50,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
1,1,1,2010-02-12,46039.49,True,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,A,151315
2,1,1,2010-02-19,41595.55,False,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,A,151315
3,1,1,2010-02-26,19403.54,False,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,A,151315
4,1,1,2010-03-05,21827.90,False,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,A,151315


In [ ]:
USE_SMALL = False

if USE_SMALL:
    df_work = df.query("Store == 1 and Dept == 1").copy()
else:
    df_work = df.copy()


In [ ]:
def create_features(df):
    df['Date'] = pd.to_datetime(df['Date'])
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['Week'] = df['Date'].dt.isocalendar().week.astype(int)
    df['Day'] = df['Date'].dt.day
    df['DayOfWeek'] = df['Date'].dt.dayofweek
    df['DayOfYear'] = df['Date'].dt.dayofyear
    df['Is_Holiday'] = df['IsHoliday'].astype(int)
    return df


In [ ]:
df_work = df_work.sort_values('Date')
df_work = create_features(df_work)


In [ ]:
feature_cols = [
    'Store', 'Dept', 'Year', 'Month', 'Week', 'Day',
    'DayOfWeek', 'DayOfYear', 'Is_Holiday',
    'Temperature', 'Fuel_Price', 'CPI', 'Unemployment'
]

In [ ]:
X = df_work[feature_cols]
y = df_work['Weekly_Sales']

In [ ]:
LAGS    = [1, 2, 3, 4, 5, 6, 7, 14, 28]
WINDOWS = [7, 14, 28]


In [ ]:
for lag in LAGS:
    df_work[f'lag_{lag}'] = (df_work
        .groupby(['Store','Dept'])['Weekly_Sales']
        .shift(lag))

for w in WINDOWS:
    df_work[f'roll_mean_{w}'] = (df_work
        .groupby(['Store','Dept'])['Weekly_Sales']
        .shift(1).rolling(w).mean())
    df_work[f'roll_std_{w}']  = (df_work
        .groupby(['Store','Dept'])['Weekly_Sales']
        .shift(1).rolling(w).std())

df_work.dropna(inplace=True)

In [ ]:
feature_cols = [
    'Store','Dept','Year','Month','Week','Day','DayOfWeek','DayOfYear',
    'Is_Holiday','Temperature','Fuel_Price','CPI','Unemployment'
] + \
    [f'lag_{l}' for l in LAGS] + \
    [f'roll_mean_{w}' for w in WINDOWS] + \
    [f'roll_std_{w}'  for w in WINDOWS]


In [ ]:

X = df_work[feature_cols]
y = df_work['Weekly_Sales']

In [ ]:
CUT_DATE = '2012-07-01'
train_mask = df_work['Date'] < CUT_DATE
X_train, y_train = X[train_mask], y[train_mask]
X_valid, y_valid = X[~train_mask], y[~train_mask]

print(X_train.shape, X_valid.shape)

(64522, 28) (30102, 28)


In [ ]:
!pip install -q xgboost mlflow

import xgboost as xgb, mlflow, mlflow.sklearn, os
from sklearn.metrics import mean_squared_error

os.environ['MLFLOW_TRACKING_URI'] = 'file:/content/mlruns'
mlflow.set_experiment('XGBoost_Training')

with mlflow.start_run(run_name='XGB_baseline_lag_roll'):

    params = dict(
        n_estimators=1500,
        learning_rate=0.03,
        max_depth=8,
        subsample=0.8,
        colsample_bytree=0.8,
        tree_method='hist',          # 'gpu_hist' თუ GPU გაქვს
        random_state=42,
        objective='reg:squarederror',
        early_stopping_rounds=100
    )

    model = xgb.XGBRegressor(**params)
    model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        verbose=200
    )

    preds = model.predict(X_valid)
    rmse  = mean_squared_error(y_valid, preds) ** 0.5
    print(f"Validation RMSE: {rmse:0.2f}")

    mlflow.log_params(params)
    mlflow.log_metric("val_rmse", rmse)
    mlflow.sklearn.log_model(model, artifact_path='model')


[0]	validation_0-rmse:22461.73392
[200]	validation_0-rmse:3616.91300
[229]	validation_0-rmse:3638.52234


2025/07/07 11:44:07 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Validation RMSE: 3561.85


2025/07/07 11:44:10 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [ ]:
!pip install pyngrok -q

In [ ]:
from pyngrok import ngrok, conf

NGROK_AUTH_TOKEN = "2zXieAERZUJhQWKxXhKwvjXc1fh_2CSAEsiYxxG6iSoupmRN9"
conf.get_default().auth_token = NGROK_AUTH_TOKEN

print("ngrok done.")

ngrok done.


In [ ]:
get_ipython().system_raw("mlflow ui --port 5000 &")

print("ngrok  MLflow UI")
public_url = ngrok.connect(addr="5000", proto="http")
print(f"MLflow UI: {public_url}")

ngrok  MLflow UI
MLflow UI: NgrokTunnel: "https://1bd1-34-61-217-101.ngrok-free.app" -> "http://localhost:5000"


In [ ]:

print(f"ave: {df_work['Weekly_Sales'].mean():.2f}")
print(f"stdev: {df_work['Weekly_Sales'].std():.2f}")
print(f"min: {df_work['Weekly_Sales'].min():.2f}")
print(f"max: {df_work['Weekly_Sales'].max():.2f}")

ave: 17968.43
stdev: 24524.41
min: -370.12
max: 630999.19
